## Dataset Breast Cancer

### Load Dataset

In [32]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer()

X = breast_cancer.data
y = breast_cancer.target

### Splitting Dataset

In [11]:
from sklearn.model_selection import train_test_split

X_train_set, X_test_set, y_train_set, y_test_set = train_test_split(X, y, test_size = 0.2, random_state = 1)

### Training Dataset
#### a. DecisionTreeClassifier
##### Training

In [19]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 10)

model.fit(X_train_set, y_train_set)

DecisionTreeClassifier(max_depth=10)

##### Evaluation

In [21]:
from sklearn.metrics import classification_report

y_prediction = model.predict(X_test_set)

print(classification_report(y_test_set, y_prediction))

              precision    recall  f1-score   support

           0       0.97      0.90      0.94        42
           1       0.95      0.99      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



#### b. Id3Estimator
##### Training

##### Evaluation

#### c. K Means 
##### Training

##### Evaluation

#### d. LogisticRegression
##### Training

##### Evaluation

#### e. Neural_network
##### Training

##### Evaluation

#### f. SVM
##### Training

##### Evaluation

## Dataset Play-tennis

### Load Dataset

In [119]:
import pandas as pd

dataset_play_tennis = pd.read_csv("PlayTennis.csv")
dataset_play_tennis_data = dataset_play_tennis[{'Outlook', 'Temperature', 'Humidity', 'Wind'}]
dataset_play_tennis_target = dataset_play_tennis[{'Play Tennis'}]

# dataset_play_tennis_target
# dataset_play_tennis_data

### Label Encoder

##### Humidity : 0 = "High", 1 = "Normal"
##### Temperature : 0 = "Cool", 1 = "Hot", 2 = "Mild"
##### Outlook : 0 = "Overcast", 1 = "Rain", 2 = "Sunny"
##### Wind : 0 = "Strong", 1 = "Weak"
##### Play Tennis : 0 = "No", 1 = "Yes"

In [116]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

x_data_encoded = dataset_play_tennis_data.copy()
if dataset_play_tennis_data is not None:
    for column in dataset_play_tennis_data.columns:
        x_data_encoded[column] = le.fit_transform(x_data_encoded[column])

y_target_encoded = dataset_play_tennis_target.copy()
if dataset_play_tennis_data is not None:
    for column in dataset_play_tennis_target.columns:
        y_target_encoded[column] = le.fit_transform(y_target_encoded[column]) 

### Splitting Data

In [120]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data_encoded, y_target_encoded, test_size=0.2, random_state=1)

### Training Dataset
#### a. DecisionTreeClassifier
##### Training

In [121]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 10)

model.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=10)

In [123]:
from sklearn.metrics import classification_report

y_prediction = model.predict(x_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3

